Reference:
https://note.nkmk.me/python-pandas-time-series-resample-asfreq/

In [1]:
import pandas as pd

## asfreq()の使い方

以下の2日おきのデータを例とする。

In [2]:
df = pd.DataFrame(
    {'value': range(1, 32, 2)},
    index=pd.date_range('2018-08-01', '2018-08-31', freq='2D')
)
print(df)

            value
2018-08-01      1
2018-08-03      3
2018-08-05      5
2018-08-07      7
2018-08-09      9
2018-08-11     11
2018-08-13     13
2018-08-15     15
2018-08-17     17
2018-08-19     19
2018-08-21     21
2018-08-23     23
2018-08-25     25
2018-08-27     27
2018-08-29     29
2018-08-31     31


## 基本的な使い方

asfreq()の第一引数freqにはD（日次）、W（週次）などの頻度コードを指定する。

上述のようにasfreq()はデータの選択なので、元のデータに無い日時の値は欠損値NaNとなる。

In [3]:
print(df.asfreq('10D'))

            value
2018-08-01      1
2018-08-11     11
2018-08-21     21
2018-08-31     31


In [4]:
print(df.asfreq('5D'))

            value
2018-08-01    1.0
2018-08-06    NaN
2018-08-11   11.0
2018-08-16    NaN
2018-08-21   21.0
2018-08-26    NaN
2018-08-31   31.0


In [5]:
print(df.asfreq('W'))

            value
2018-08-05    5.0
2018-08-12    NaN
2018-08-19   19.0
2018-08-26    NaN


In [6]:
print(df.asfreq('W-WED'))

            value
2018-08-01    1.0
2018-08-08    NaN
2018-08-15   15.0
2018-08-22    NaN
2018-08-29   29.0


以下、元のデータに無い日時の値の穴埋め方法について説明する。asfreq()でできるのは任意の値や前後の値をそのまま使った単純な穴埋め。前後の値を使った線形補間などはresample()を使う。後述。

## 元データに無い場合に任意の値で穴埋め: 引数fill_value

元のデータに無い日時の値を任意の値で穴埋めするには引数fill_valueを指定する。

In [7]:
print(df.asfreq('W', fill_value=0))

            value
2018-08-05      5
2018-08-12      0
2018-08-19     19
2018-08-26      0


## 元データに無い場合に前後の値で穴埋め: 引数method

元のデータに無い日時の値を前後の値で穴埋めするには引数methodを指定する。

method=padまたはmethod=ffillで前の値で穴埋め、method=backfillまたはmethod=bfillで後ろの値で穴埋めとなる。

In [8]:
print(df.asfreq('W', method='pad'))

            value
2018-08-05      5
2018-08-12     11
2018-08-19     19
2018-08-26     25


In [9]:
print(df.asfreq('W', method='ffill'))

            value
2018-08-05      5
2018-08-12     11
2018-08-19     19
2018-08-26     25


In [10]:
print(df.asfreq('W', method='backfill'))

            value
2018-08-05      5
2018-08-12     13
2018-08-19     19
2018-08-26     27


In [11]:
print(df.asfreq('W', method='bfill'))

            value
2018-08-05      5
2018-08-12     13
2018-08-19     19
2018-08-26     27


欠損値NaNが連続している場合も同様。

In [12]:
df_3D = pd.DataFrame(
    {'value': range(1, 32, 3)},
    index=pd.date_range('2018-08-01', '2018-08-31', freq='3D')
)

print(df_3D.asfreq('D', method='bfill'))

            value
2018-08-01      1
2018-08-02      4
2018-08-03      4
2018-08-04      4
2018-08-05      7
2018-08-06      7
2018-08-07      7
2018-08-08     10
2018-08-09     10
2018-08-10     10
2018-08-11     13
2018-08-12     13
2018-08-13     13
2018-08-14     16
2018-08-15     16
2018-08-16     16
2018-08-17     19
2018-08-18     19
2018-08-19     19
2018-08-20     22
2018-08-21     22
2018-08-22     22
2018-08-23     25
2018-08-24     25
2018-08-25     25
2018-08-26     28
2018-08-27     28
2018-08-28     28
2018-08-29     31
2018-08-30     31
2018-08-31     31


前後の値から線形補間などを行いたい場合はresample()を使う。後述。

## 重複データがある場合

以下のような同じ日付の違う時刻のデータを例とする。

In [14]:
df_h = pd.DataFrame(
    {'value': range(9)},
    index=pd.date_range('2018-08-01', '2018-08-05', freq='12h')
)
print(df_h)

                     value
2018-08-01 00:00:00      0
2018-08-01 12:00:00      1
2018-08-02 00:00:00      2
2018-08-02 12:00:00      3
2018-08-03 00:00:00      4
2018-08-03 12:00:00      5
2018-08-04 00:00:00      6
2018-08-04 12:00:00      7
2018-08-05 00:00:00      8


これをasfreq()でfreq='D'（日次）としてダウンサンプリングする場合、先頭の値が選択される。

In [15]:
print(df_h.asfreq('D'))

            value
2018-08-01      0
2018-08-02      2
2018-08-03      4
2018-08-04      6
2018-08-05      8


平均値など、重複する複数のデータを集約した値を使いたい場合は次に説明するresample()を使う。

In [16]:
print(df_h.resample('D').mean())

            value
2018-08-01    0.5
2018-08-02    2.5
2018-08-03    4.5
2018-08-04    6.5
2018-08-05    8.0


## resample()の使い方

ここでも2日おきのデータを例とする。

In [17]:
df = pd.DataFrame(
    {'value': range(1, 32, 2)},
    index=pd.date_range('2018-08-01', '2018-08-31', freq='2D'))

print(df)

            value
2018-08-01      1
2018-08-03      3
2018-08-05      5
2018-08-07      7
2018-08-09      9
2018-08-11     11
2018-08-13     13
2018-08-15     15
2018-08-17     17
2018-08-19     19
2018-08-21     21
2018-08-23     23
2018-08-25     25
2018-08-27     27
2018-08-29     29
2018-08-31     31


補間してアップサンプリングする例については後述。

## 基本的な使い方

resample()の第一引数ruleにも、asfreq()と同様にD（日次）、W（週次）などの頻度コードを指定する。

resample()が返すのはDatetimeIndexResampler型のオブジェクトで、それ自体をprint()で出力しても値は表示されない。

In [19]:
print(df.resample('W'))
print(type(df.resample('W')))

DatetimeIndexResampler [freq=<Week: weekday=6>, axis=0, closed=right, label=right, convention=start, origin=start_day]
<class 'pandas.core.resample.DatetimeIndexResampler'>


mean()（平均値）やmedian()（中央値）、sum()（合計）などのメソッドを呼ぶことで集約された値が算出される。以下は週ごとの平均などの例。